In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import nltk
from nltk.corpus import stopwords, names
import string
import joblib
import scipy as sp
import textwrap
import csv
import random

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, Input
import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import  classification_report
import json

In [3]:
dataset_url = "https://raw.githubusercontent.com/JULIELab/EmoBank/master/corpus/emobank.csv"
df_data = pd.read_csv(dataset_url,quoting = 0)
corpus =  df_data.iloc[:, [5]]
vad = df_data.iloc[:, [2,3,4]]

In [4]:
data_file = open('/content/intents.json').read()
questions = json.loads(data_file)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tf_id_array = vectorizer.fit_transform(corpus['text'].tolist()).toarray()

In [41]:
import nltk
nltk.download('stopwords')
nltk.download('names')

from nltk.corpus import stopwords, names
import string
import re
my_stopwords = set(stopwords.words("english") + names.words() + ["''", "'d", "'ll", "'re", "'ve", '--', '...', '1', '10', '2', '20', '3', '4', '``', '–', '—', '’', '“', '”'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


In [8]:
def thorough_filter(words):
    filtered_words = []
    regexp = re.compile('[0-9]+')
    for word in words:
      if not regexp.search(word):
        pun = []
        for letter in word:
          pun.append(letter in string.punctuation.strip('!').strip('?'))
        if not all(pun):
          filtered_words.append(word)
    return filtered_words

filtered = thorough_filter(vectorizer.get_feature_names_out())

In [9]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('rslp')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [36]:
from nltk.stem.snowball import SnowballStemmer
class SnowballTokenizer(object):
    def __call__(self, text):
        words = [word for word in nltk.word_tokenize(text) if (word not in my_stopwords)]
        words = thorough_filter(words)
        return [SnowballStemmer("english").stem(word) for word in words]

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer=SnowballTokenizer(), max_df=0.95, min_df=3, sublinear_tf=True)
tf_id_array = vectorizer.fit_transform(corpus['text'].tolist()).toarray()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [11]:
splits = np.array(df_data.iloc[:,1].tolist())
train_set = tf_id_array[splits=='train']
train_labels = vad[splits=='train'].to_numpy()
test_set = tf_id_array[splits=='test']
test_labels = vad[splits=='test'].to_numpy()
dev_set = tf_id_array[splits=='dev']
dev_labels = vad[splits=='dev'].to_numpy()

sample_length = train_set.shape[1]

In [39]:
emotion_coeff = {
  'anger': np.array([-0.43, 0.67, 0.34]),
  'joy': np.array([0.76, 0.48, 0.35]),
  'surpise': np.array([0.4, 0.67, -0.13]),
  'disgust': np.array([-0.6, 0.35, 0.11]),
  'fear': np.array([-0.64, 0.6, -0.43]),
  'sadness': np.array([-0.63, 0.27, -0.33])
}

vad2emotion = {
    0: 'anger',
    1: 'joy',
    2: 'surprise',
    3: 'disgust',
    3: 'fear',
    4: 'sadness'
}

In [13]:
for k,v in emotion_coeff.items():
  emotion_coeff[k] = 2*v+3

emo_coeff = np.array(list(emotion_coeff.values()))

neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(emo_coeff)

train_labels_emo_dst, train_labels_emo = neigh.kneighbors(train_labels)
test_labels_emo_dst, test_labels_emo = neigh.kneighbors(test_labels)

In [14]:
def summarize_diagnostics(history):
	plt.figure(figsize=(8, 8))
	plt.suptitle('Training Curves')
	# plot loss
	plt.subplot(211)
	plt.title('Loss')
	plt.plot(history.history['loss'], color='blue', label='train')
	plt.plot(history.history['val_loss'], color='orange', label='val')
	plt.legend(loc='upper right')
	# plot accuracy
	plt.subplot(212)
	plt.title('Accuracy')
	plt.plot(history.history['accuracy'], color='blue', label='train')
	plt.plot(history.history['val_accuracy'], color='orange', label='val')
	plt.legend(loc='lower right')
	return plt

In [18]:
def model_evaluation(model, evaluation_steps, test_set, test_labels):
  print('\nTest set evaluation metrics')
  print(model.evaluate(test_set, steps = evaluation_steps))

def model_report(model, history, test_set, test_labels, evaluation_steps = 10):
  plt = summarize_diagnostics(history)
  plt.show()
  model_evaluation(model, evaluation_steps, test_set, test_labels)

def train_model(model, train_set, train_labels, val_set, val_labels, epochs = 10, steps_per_epoch = 2, validation_steps = 1, tensorboard_entry = False, early_stopping = False, learning_rate = False):

  log_dir = "./logs/fit/" + model.name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  callbacks = []
  if tensorboard_entry:
    callbacks.append(tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1))
  if early_stopping:
    callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta = 0 , patience = 40, restore_best_weights= True))
  if learning_rate:
      callbacks.append(tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=10, min_lr=0.00001))

  history = model.fit(train_set, train_labels, epochs=epochs, steps_per_epoch=steps_per_epoch, validation_data=(val_set, val_labels), validation_steps=validation_steps, callbacks = callbacks)
  return(history)

In [34]:
load = True

def init_zuccbot():

  model = models.Sequential(name="zuccbot")
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(32, activation='relu'))
  model.add(layers.Dense(16, activation='relu'))
  model.add(layers.Dense(3))

  model.compile(optimizer="rmsprop", loss='mean_squared_error', metrics=['accuracy'])

  return model

if not load:
  model = init_zuccbot()
  history = train_model(model, train_set, train_labels, dev_set, dev_labels, 300, 30, 5)
  model.save("chatbob-model")
else:
  model = init_zuccbot()
  model = models.load_model("chatbob-model")

In [30]:
if load:
  reconstructed_model = models.load_model("/content/chatbob-model")

def analyseSentence(sentence):
  inp = vectorizer.transform([sentence]).toarray()
  vad_pred = model.predict(inp)
  emo_dst, emo_pred = neigh.kneighbors(vad_pred, 3)
  emos = [vad2emotion[x] for x in emo_pred.ravel()]
  return 1/emo_dst.ravel(), emos

def analyseQuestionnaire(answers):
  emotions = ['anger', 'joy', 'surprise', 'fear', 'sadness']
  emo_vals = {x:0 for x in emotions}
  for sen in answers:
    emo_ints, emos = analyseSentence(sen)
    for emo, emo_int in zip(emos, emo_ints):
      emo_vals[emo] += emo_int
  emo_vals = { emo: emo_int/sum(emo_vals.values()) for emo, emo_int in emo_vals.items()}
  return emo_vals

In [28]:
import nltk
nltk.download('stopwords')
nltk.download('names')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('rslp')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [50]:
sentence = "Hello, how are you"
emotion_intensity, emotions = analyseSentence(sentence)
print(f"Emotion Intensity: {emotion_intensity}, Emotions: {emotions}")

questionnaire_answers = ["You are so dumb, get out!"]
questionnaire_emotions = analyseQuestionnaire(questionnaire_answers)
print("Emotion Distribution in Questionnaire:")
for emotion, intensity in questionnaire_emotions.items():
    print(f"{emotion}: {intensity}")

1/1 [==============================] - 0s 70ms/step
Emotion Intensity: [0.70068466 0.58897946 0.58320214], Emotions: ['sadness', 'anger', 'surprise']
1/1 [==============================] - 0s 74ms/step
Emotion Distribution in Questionnaire:
anger: 0.548051637807054
joy: 0.0
surprise: 0.0
fear: 0.20719231603082588
sadness: 0.24475604616212002
